In [1]:
# o use the latest publish `kubeflow-metadata` library, you can run:
!pip install kubeflow-metadata --user
# Install other packages:
!pip install pandas --user
# Then restart the Notebook kernel.


import pandas
from kubeflow.metadata import metadata
from datetime import datetime
from uuid import uuid4

METADATA_STORE_HOST = "metadata-grpc-service.kubeflow" # default DNS of Kubeflow Metadata gRPC serivce.
METADATA_STORE_PORT = 8080


ws1 = metadata.Workspace(
    # Connect to metadata service in namespace kubeflow in k8s cluster.
    store=metadata.Store(grpc_host=METADATA_STORE_HOST, grpc_port=METADATA_STORE_PORT),
    name="Mnist Workspace",
    description="Artifact on Mnist Workspace",
    labels={"n1": "v1"})


r = metadata.Run(
    workspace=ws1,
    name="run-" + datetime.utcnow().isoformat("T") ,
    description="A run in Mnsit workspace",
)

from uuid import uuid4


def add_suffix(name: str) -> str:
    """
    Appends an underscore and hexidecimal UUID to `name`

    :param str name: String to be suffixed
    :return: Suffixed string
    :rtype: str
    """
    return f"{name}_{uuid4().hex}"

run = metadata.Run(
    workspace=ws1,
    name=add_suffix("run"),
    description="A run in our workspace",
)


exec = metadata.Execution(
    name = "execution" + datetime.utcnow().isoformat("T") ,
    workspace=ws1,
    run=r,
    description="Mnsit execution example",
)
print("An execution was created with id %s" % exec.id)


date_set_version = "data_set_version_" + str(uuid4())
data_set = exec.log_input(
        metadata.DataSet(
            description="an example data",
            name="mytable-dump",
            owner="owner@my-company.org",
            uri="http://minio-service.kubeflow:9000/datastore2/datasets.tar.gz",
            version=date_set_version,
            query=""))
print("Data set id is {0.id} with version '{0.version}'".format(data_set))

model_version = "model_version_" + str(uuid4())
model = exec.log_output(
    metadata.Model(
            name="MNIST",
            description="model to recognize handwritten digits",
            owner="someone@kubeflow.org",
            uri="http://minio-service.kubeflow:9000/datastore2/datasets.tar.gz",
            model_type="neural network",
            training_framework={
                "name": "tensorflow",
                "version": "v1.0"
            },
             hyperparameters={
            "learning_rate": 0.5,
            "layers": [10, 3, 1],
            "early_stop": True,
            },
            version=model_version,
            labels={"mylabel": "l1"}))
print(model)
print("\nModel id is {0.id} and version is {0.version}".format(model))



metrics = exec.log_output(
    metadata.Metrics(
            name="MNIST-evaluation",
            description="validating the MNIST model",
            owner="mnist@kubeflow.org",
            uri="http://minio-service.kubeflow:9000/datastore2/datasets.tar.gz",
            data_set_id=str(data_set.id),
            model_id=str(model.id),
            metrics_type=metadata.Metrics.VALIDATION,
            values={"accuracy": 0.95},
            labels={"mylabel": "l1"}))

print("Metrics id is %s" % metrics.id)


app = metadata.Execution(
    name="Serving the MNIST model",
    workspace=ws1,
    description="An execution to represent the model serving component",
)

served_model = metadata.Model(
    name="MNIST",
    uri="http://minio-service.kubeflow:9000/mnist/mnist",
    version=model.version,
)

m = app.log_input(served_model)

print(f"Serving model with ID:      {m.id}")
print(f"Serving model with version: {m.version}")


artifacts = ws1.list(metadata.Model.ARTIFACT_TYPE_NAME)
artifacts


import pandas as pd

pd.DataFrame.from_dict(artifacts)


model_events = ws1.store.get_events_by_artifact_ids([model.id])

execution_ids = set(e.execution_id for e in model_events)
print(f"Executions related to the model: {execution_ids}")

trainer_events = ws1.store.get_events_by_execution_ids([exec.id])
artifact_ids = set(e.artifact_id for e in trainer_events)
print(f"Artifacts related to the trainer: {artifact_ids}")

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
An execution was created with id 19
Data set id is 31 with version 'data_set_version_aeb4c6cd-bd0f-4d38-ba7c-e0e4d3f42b64'
kubeflow.metadata.metadata.Model(workspace=None, name='MNIST', description='model to recognize handwritten digits', owner='someone@kubeflow.org', uri='http://minio-service.kubeflow:9000/datastore2/datasets.tar.gz', version='model_version_199cc53e-65f6-4ebb-948a-7fcaccdb8f4f', model_type='neural network', training_framework={'name': 'tensorflow', 'version': 'v1.0'}, hyperparameters={'learning_rate': 0.5, 'layers': [10, 3, 1], 'early_stop': True}, labels={'mylabel': 'l1'}, id=32, create_time='2021-03-18T06:41:45.232839Z', kwargs={})

Model id is 32 and version is model_version_199cc53e-65f6-4ebb-948a-7fcaccdb8f4f
Metrics id is 33
Serving model with ID:      34
Serving model with v